# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [45]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [46]:
# Create the path to open the file
weatherpy_data = "../WeatherPy/cities.csv"

In [47]:
# Read the file
cities_temperature = pd.read_csv(weatherpy_data)

# Organize file headers
cities_temperature = cities_temperature[["City", "Lat", "Lng", "Max Temp", "Humidity", "Cloudiness", "Wind Speed", "Country", "Date"]]
cities_temperature.dropna(inplace = True)
cities_temperature.head()

City    Lat     Lng  Max Temp  Humidity  Cloudiness  Wind Speed  \
0     new norfolk -42.78  147.06     61.00        65           0        3.00   
1         kaniama  -7.57   24.18     68.50        79           4        1.83   
2  vestmannaeyjar  63.44  -20.27     43.61        80         100        2.93   
3   pacific grove  36.62 -121.92     63.00        87          40        3.67   
4     barentsburg  43.32  145.57     62.60        59          20        8.05   

  Country        Date  
0      AU  1602036388  
1      CD  1602036390  
2      IS  1602036392  
3      US  1602036229  
4      JP  1602036399

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [48]:
# Store latitude and longitude in locations
locations = cities_temperature[["Lat", "Lng"]]

# Store humidity values into a series to use it as weight 
humidity = cities_temperature["Humidity"].astype(float)

In [49]:
# Plot Heatmap
fig = gmaps.figure(map_type="TERRAIN")
fig = gmaps.figure(center=(0, 0), zoom_level=2.25)
# Create a heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)

# Add layer
fig.add_layer(heat_layer)

# Display figure 
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [50]:
# Narrow the data frame
narrow_cities_data = cities_temperature.loc[((cities_temperature["Max Temp"]<=85.00) & (cities_temperature["Max Temp"]>73.00) & (cities_temperature["Wind Speed"]<10.00) & (cities_temperature["Cloudiness"]==0))]
narrow_cities_data


City    Lat    Lng  Max Temp  Humidity  Cloudiness  Wind Speed  \
45        padang -15.86 -38.88     75.20        83           0        5.82   
55    chokurdakh  31.26  32.28     78.80        78           0        6.93   
80        tiznit   0.49  42.79     74.79        89           0        3.62   
111          obo -21.64 -41.05     75.02        90           0        1.16   
128  mys shmidta -10.41 -36.43     74.03        82           0        7.72   
142  port alfred  22.57  59.53     77.74        54           0        5.79   
143       sabang  21.43  39.83     78.57        70           0        1.25   
205      dunedin  14.90 -24.50     78.60        73           0        4.32   
208       samusu  24.69  46.72     77.00        25           0        5.82   
277       narsaq  27.15  57.08     80.60        83           0        2.24   
328   walvis bay  16.47 -98.42     75.63        91           0        1.72   
355        sucua -13.28 -50.16     83.53        22           0        3.42   
463      meyungs -11.09 -43.14     75.63        48           0        5.88   
513        genhe -15.97 -54.97     83.80        24           0        3.83   

    Country        Date  
45       BR  1602036492  
55       EG  1602036515  
80       SO  1602036573  
111      BR  1602036550  
128      BR  1602036682  
142      OM  1602036712  
143      SA  1602036714  
205      CV  1602036863  
208      SA  1602036694  
277      IR  1602037028  
328      MX  1602037147  
355      BR  1602037211  
463      BR  1602037466  
513      BR  1602037585

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [82]:
#Store into variable named hotel_df
hotel_df = narrow_cities_data[["City", "Lat", "Lng", "Humidity", "Wind Speed", "Country"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""
hotel_df

/Users/manuelamunoz/opt/anaconda3/envs/PythonData2/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


City    Lat    Lng  Humidity  Wind Speed Country Hotel Name
45        padang -15.86 -38.88        83        5.82      BR           
55    chokurdakh  31.26  32.28        78        6.93      EG           
80        tiznit   0.49  42.79        89        3.62      SO           
111          obo -21.64 -41.05        90        1.16      BR           
128  mys shmidta -10.41 -36.43        82        7.72      BR           
142  port alfred  22.57  59.53        54        5.79      OM           
143       sabang  21.43  39.83        70        1.25      SA           
205      dunedin  14.90 -24.50        73        4.32      CV           
208       samusu  24.69  46.72        25        5.82      SA           
277       narsaq  27.15  57.08        83        2.24      IR           
328   walvis bay  16.47 -98.42        91        1.72      MX           
355        sucua -13.28 -50.16        22        3.42      BR           
463      meyungs -11.09 -43.14        48        5.88      BR           
513        genhe -15.97 -54.97        24        3.83      BR

In [88]:
# Base_url using nearbysearch

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Define params

params = {
    "location": target_cordinates,
    "radius": 5000,
    "types": "lodging",
    "keyword": "hotel",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    hotel_name = requests.get(base_url, params=params).json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print(f"City {index} was not found")

/Users/manuelamunoz/opt/anaconda3/envs/PythonData2/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


City 80 was not found


In [92]:
# Drop the city not found
hotel_df.sort_values

City    Lat    Lng  Humidity  Wind Speed Country  \
45        padang -15.86 -38.88        83        5.82      BR   
55    chokurdakh  31.26  32.28        78        6.93      EG   
111          obo -21.64 -41.05        90        1.16      BR   
128  mys shmidta -10.41 -36.43        82        7.72      BR   
142  port alfred  22.57  59.53        54        5.79      OM   
143       sabang  21.43  39.83        70        1.25      SA   
205      dunedin  14.90 -24.50        73        4.32      CV   
208       samusu  24.69  46.72        25        5.82      SA   
277       narsaq  27.15  57.08        83        2.24      IR   
328   walvis bay  16.47 -98.42        91        1.72      MX   
355        sucua -13.28 -50.16        22        3.42      BR   
463      meyungs -11.09 -43.14        48        5.88      BR   
513        genhe -15.97 -54.97        24        3.83      BR   

                                  Hotel Name  
45                          Pousada Belmonte  
55   فندق رستا بورسعيد Resta Port Said Hotel  
111                   POUSADA SOBRE AS ÁGUAS  
128                           Pousada Raizes  
142                          Sur Plaza Hotel  
143               فندق زمازم - Zamazim Hotel  
205   Hotel Ocean View & Restaurante Seafood  
208           Best Western Plus Fursan Hotel  
277                      Minab Tourism Hotel  
328                       Hotel Costa Dorada  
355                Hotel Veredas do Araguaia  
463                       Hotel Barra Palace  
513                               Taba Hotel

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
